python 3.8 + pytorch 1.8

# 目的：搭建seq2seq+attention模型并进行模拟语句预测
- 输入：id序列(input)
- 输出：预测id序列(decoder_output)
- 主要步骤：
 1. 构建编码器
 2. 构建译码器，并加入attention机制
 3. 编码器译码器模型初始化
 4. 训练及测试

## 1. 构建基于GRU的编码器

In [ ]:
import torch.nn as nn

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size  # 隐层大小
        self.embedding = nn.Embedding(input_size, hidden_size)  # 词向量层
        self.gru = nn.GRU(hidden_size, hidden_size)  # GRU 层

    def forward(self, input, hidden):
        # 输入的 batch_size 为 1，且每次输入一个单词
        embedded = self.embedding(input).view(1, 1, -1)  # 词向量计算，注意输出后的维度变化
        output, hidden = self.gru(embedded, hidden)  # GRU 层计算
        return output, hidden

    def initHidden(self):  # 初始化隐状态
        return torch.zeros(1, 1, self.hidden_size)

## 2. 构建基于 GRU 的解码器，并且加入 attention 机制

In [ ]:
# forward过程：

import torch.nn.functional as F

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size  # 隐层大小
        self.output_size = output_size  # 输出层大小，实际上为目标语言的词典大小

        self.embedding = nn.Embedding(
            self.output_size, self.hidden_size)  # 词向量层
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)  # GRU 层
        self.out = nn.Linear(self.hidden_size*2, self.output_size)  # 输出层

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)  # 输入当前单词的词向量计算
        embedded = F.relu(embedded)  # 激活函数 Relu
        output, hidden = self.gru(embedded, hidden)  # GRU 运算

        # 注意力运算，由 encoder_outputs 和 hidden 进行内积计算
        attn_weights = F.softmax(torch.bmm(encoder_outputs.unsqueeze(
            0), hidden.view(1, self.hidden_size, -1)), dim=1)
        # 添加注意力机制的源文信息，获得加权和后的向量表征
        weighted_context = torch.matmul(
            attn_weights.squeeze(2), encoder_outputs)
        # 拼接当前信息以及添加注意力机制的源文信息
        output = torch.cat([output.squeeze(0), weighted_context], dim=1)
        # 对输出进行 softmax 计算
        output = F.softmax(self.out(output), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):  # 初始化隐状态
        return torch.zeros(1, 1, self.hidden_size)

## 3. 初始化编码器及解码器

In [ ]:
INPUT_SIZE = 1000  # 输入维度，实际上为源语言的词典大小
HIDDEN_SIZE = 60  # 隐层大小
OUTPUT_SIZE = 1000  # 输出维度，实际上为目标语言的词典大小

# 初始化编码器及解码器
my_encoder = EncoderRNN(input_size=INPUT_SIZE, hidden_size=HIDDEN_SIZE)
my_decoder = AttnDecoderRNN(hidden_size=HIDDEN_SIZE, output_size=OUTPUT_SIZE)
my_encoder,my_decoder

## 4. 训练及测试

In [ ]:
# 输入假设的源语句，进行编码过程
import torch

# 假设已转换为 id 的输入源语句如下
input = torch.tensor([[0], [4], [3], [1]])
# 初始化 encoder 隐状态
encoder_hidden = my_encoder.initHidden()
# 获取输入文本长度
input_length = input.size(0)
# 用于存在 encoder 每一步的输出
encoder_outputs = torch.zeros(input_length, my_encoder.hidden_size)
# 每一步进行 rnn(GRU) 运算，并将结果保存，得到 encoder_outputs
for i in range(input_length):
    encoder_output, encoder_hidden = my_encoder(input[i], encoder_hidden)
    encoder_outputs[i] = encoder_output[0, 0]

encoder_outputs.shape # shape = [input_length, hidden_size]

In [5]:
# decoder 首输入为 "<s>"，假设其对应 id 为 0
decoder_input = torch.tensor([[0]])
# encoder 隐状态传给 decoder，作为其初始隐状态
decoder_hidden = encoder_hidden
# 进行一步编码过程
decoder_output, decoder_hidden, decoder_attention = my_decoder(
    decoder_input, decoder_hidden, encoder_outputs)
decoder_output.shape  # [1,output_size], 在输出结果中选概率最高者作为编码出的单词
# TODO：重复上一步进行编码
# 最终编码输出是在 output_size 维中选取概率最大的单词作为所编码出的词汇，
# 接着将其作为下一步输入，再进入下一步的编码过程

torch.Size([1, 1000])